In [1]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']


twenty_train = fetch_20newsgroups(subset='train',
categories=categories, shuffle=True, random_state=42)





In [2]:
twenty_train.target_names


['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [3]:
len(twenty_train.data)
len(twenty_train.filenames)


2257

In [4]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))
print(twenty_train.target_names[twenty_train.target[0]])

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
comp.graphics


In [5]:
twenty_train.target[:10]


array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2], dtype=int64)

In [6]:
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t])


comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape


(2257, 35788)

In [8]:
count_vect.vocabulary_.get(u'algorithm')


4690

# 从出现次数到出现频率


In [9]:

from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape


(2257, 35788)

In [10]:


tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape


(2257, 35788)

# 训练分类器

In [11]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [12]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))



'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


# 构建 Pipeline（管道）

In [ ]:
from sklearn.pipeline import Pipeline
    text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])

# 利用sklearn做自然语言处理（NLP）——词向量特征构建
词袋（bags of words）
用一个固定的整数id来标记在所有文档中出现的所有单词，形成字典
对于#i 个文档，计算每个单词w在文档中出现的次数n，并将n存放在X[i, id] 中。
构建词袋的过程中会遇到的问题
一般feature的数量都能达到100,000，这是如果有10,000个样本的话就需要相当大的内存。但实际上，样本的特征向量X[i] 有很多特征都是零（称之为稀疏矩阵）。

这时候需要使用scipy.sparse来存储这样的数据结构。luckily，scipy和sklearn都是同一家项目底下的，所以毫无疑问地互相兼容了。

用sklearn来对文本的每个单词做标记
在构建词袋的过程中，构建字典是很重要的一步。

将文本数值化在sklearn中用到的是CountVectorizer
导入方式如下


In [10]:
import jieba
#   训练样本如下
corpus = [
     '今天，天气不错，挺风和日丽的，上周天气不行',
     '我们下午没有课。于是我背上小书包，就去上自习了',
     '在北京大学的图书馆，打开课本，从天气知识一直看到计算机编程',
   ]
train_data = list();
for each in corpus:
    train_data.append(' '.join(jieba.cut(each)));

In [11]:
print(train_data)

['今天 ， 天气 不错 ， 挺 风和日丽 的 ， 上周 天气 不行', '我们 下午 没有 课 。 于是 我 背上 小 书包 ， 就 去 上 自习 了', '在 北京大学 的 图书馆 ， 打开 课本 ， 从 天气 知识 一直 看到 计算机 编程']


In [12]:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train_data)
print(vectorizer.get_feature_names())


print(X.toarray())

vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X2 = vectorizer2.fit_transform(train_data)
print(vectorizer2.get_feature_names())

print(X2.toarray())


['一直', '上周', '下午', '不行', '不错', '书包', '于是', '今天', '北京大学', '图书馆', '天气', '我们', '打开', '没有', '看到', '知识', '编程', '背上', '自习', '计算机', '课本', '风和日丽']
[[0 1 0 1 1 0 0 1 0 0 2 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 1 1 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0]
 [1 0 0 0 0 0 0 0 1 1 1 0 1 0 1 1 1 0 0 1 1 0]]
['一直 看到', '上周 天气', '下午 没有', '不错 风和日丽', '书包 自习', '于是 背上', '今天 天气', '北京大学 图书馆', '图书馆 打开', '天气 不行', '天气 不错', '天气 知识', '我们 下午', '打开 课本', '没有 于是', '看到 计算机', '知识 一直', '背上 书包', '计算机 编程', '课本 天气', '风和日丽 上周']
[[0 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1]
 [0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 1 1 0 0 1 0 1 0 1 1 0 1 1 0]]
